<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#判定贷款用户是否逾期" data-toc-modified-id="判定贷款用户是否逾期-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>判定贷款用户是否逾期</a></span></li><li><span><a href="#数据集预览" data-toc-modified-id="数据集预览-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>数据集预览</a></span></li><li><span><a href="#特征处理" data-toc-modified-id="特征处理-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>特征处理</a></span></li><li><span><a href="#数据划分" data-toc-modified-id="数据划分-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>数据划分</a></span></li><li><span><a href="#模型训练" data-toc-modified-id="模型训练-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>模型训练</a></span></li><li><span><a href="#模型评估" data-toc-modified-id="模型评估-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>模型评估</a></span></li><li><span><a href="#遇到的问题" data-toc-modified-id="遇到的问题-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>遇到的问题</a></span></li><li><span><a href="#Reference" data-toc-modified-id="Reference-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Reference</a></span></li></ul></div>

# 判定贷款用户是否逾期

给定金融数据，预测贷款用户是否会逾期。
（status是标签：0表示未逾期，1表示逾期。）

**Task1** - 构建逻辑回归模型进行预测（在构建部分数据需要进行缺失值处理和数据类型转换，如果不能处理，可以直接暴力删除）

# 数据集预览

In [1]:
import pandas as pd
data = pd.read_csv('data.csv')
print(data.shape)
data.head()

(4754, 90)


,Unnamed: 0,custid,trade_no,bank_card_no,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
0,5,2791858,20180507115231274000000023057383,卡号1,0.01,0.99,0,0.90,0.55,0.313,...,2900.0,1688.0,1200.0,75.0,1.0,2.0,1200.0,1200.0,12.0,18.0
1,10,534047,20180507121002192000000023073000,卡号1,0.02,0.94,2000,1.28,1.00,0.458,...,3500.0,1758.0,15100.0,80.0,5.0,6.0,22800.0,9360.0,4.0,2.0
2,12,2849787,20180507125159718000000023114911,卡号1,0.04,0.96,0,1.00,1.00,0.114,...,1600.0,1250.0,4200.0,87.0,1.0,1.0,4200.0,4200.0,2.0,6.0
3,13,1809708,20180507121358683000000388283484,卡号1,0.00,0.96,2000,0.13,0.57,0.777,...,3200.0,1541.0,16300.0,80.0,5.0,5.0,30000.0,12180.0,2.0,4.0
4,14,2499829,20180507115448545000000388205844,卡号1,0.01,0.99,0,0.46,1.00,0.175,...,2300.0,1630.0,8300.0,79.0,2.0,2.0,8400.0,8250.0,22.0,120.0


In [33]:
# data.columns

# 特征处理


> 无用特征删除、字符型特征编码和缺失值填充。

<font color='blue'>
**删除无用特征**

In [3]:
data.drop(['Unnamed: 0', 'custid', 'trade_no', 'bank_card_no', 'source', 'id_name'], axis=1, inplace=True)
print(data.shape)

(4754, 84)


**日期特征**（暂时删除, 以后再处理。）

In [4]:
data.drop(['first_transaction_time', 'latest_query_time', 'loans_latest_time'], axis=1, inplace=True)

<font color='blue'>
**字符型特征-编码**

In [5]:
data['reg_preference_for_trad'].value_counts()

一线城市    3403
三线城市    1064
境外       150
二线城市     131
其他城市       4
Name: reg_preference_for_trad, dtype: int64

In [6]:
dic = {}
for i, val in enumerate(list(data['reg_preference_for_trad'].unique())):
    dic[val] = i

In [7]:
data['reg_preference_for_trad'] = data['reg_preference_for_trad'].map(dic)

<font color='blue'>
**缺失特征**

In [8]:
for feature in data.columns:
    summ = data[feature].isnull().sum()
    if summ*100/4754 > 50:
        print(feature, '--', summ*100/4754)

student_feature -- 63.0626840555


1) student_feature 缺失占比多, 需要用**众数**填充;

In [9]:
data['student_feature'].value_counts()

1.0    1754
2.0       2
Name: student_feature, dtype: int64

In [10]:
data['student_feature'].fillna(1.0, inplace = True)

In [11]:
data['student_feature'].value_counts()

1.0    4752
2.0       2
Name: student_feature, dtype: int64

2) 其余特征用**均值**填充。

In [13]:
for feature in data.columns:
    summ = data[feature].isnull().sum()
    if summ:
        print(feature)
        data[feature].fillna(data[feature].mean(), inplace = True)

> 'Unnamed: 0', 'custid', 'trade_no', 'bank_card_no', 'source'都没用, 'id_name'表示姓名, 没用？

> 'take_amount_in_later_12_month_highest'数值跨度可能较大, 归一化？

> 某些特征可能包含缺失值NAN

> 'reg_preference_for_trad'需要编码

> 'latest_query_time', 'loans_latest_time'为日期特征

# 数据划分

In [15]:
features = [x for x in data.columns if x not in ['status']]

In [16]:
# 划分训练集测试集
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

X = data[features]
y = data.status

# 特征归一化
std = StandardScaler()
X = std.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split = train_test_split(X, y, test_size=0.3,random_state=2333)

# 模型训练

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [24]:
lr = LogisticRegression()
lr_param = {'penalty': ('l1', 'l2'), 'C': [0.0001,0.001,0.01,0.1,1,10,100,1000]}

grid = GridSearchCV(lr, param_grid = lr_param, cv = 5)
grid.fit(X_train, y_train)

print(grid.best_params_, grid.best_score_)

{'C': 0.1, 'penalty': 'l1'} 0.788698527202


In [29]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

# 模型评估

In [30]:
from sklearn.metrics import accuracy_score, f1_score

In [31]:
# 准确性
y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)

print('准确性：')
print('训练集：{:.4f}'.format(accuracy_score(y_train, y_train_pred)), end = ' ')
print('测试集：{:.4f}'.format(accuracy_score(y_test, y_test_pred)))

准确性：
训练集：0.7995 测试集：0.8024


In [32]:
# f1_score
y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)

print('f1_score：')
print('训练集：{:.4f}'.format(f1_score(y_train, y_train_pred)), end = ' ')
print('测试集：{:.4f}'.format(f1_score(y_test, y_test_pred)))

f1_score：
训练集：0.4677 测试集：0.4639


# 遇到的问题

1) pd.read_csv读取文件时，utf-8的编码问题。

解决方法：reference1

# Reference

1) [python问题--UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte](https://blog.csdn.net/u012429555/article/details/80960985)